# LangChain 실습

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(len(OPENAI_API_KEY))

164


In [ ]:
# !pip install openai langchain langchain-openai langchain-experimental langchainhub


   -------------------------- ------------- 2/3 [langchain-experimental]
   ---------------------------------------- 3/3 [langchain-experimental]



## 5-2. Agents

### Agents 사용 예

In [4]:
import langchain
langchain.verbose = True # 내부적으로 어떤 일이 벌어지고 있는지 상세한 **로그(Log)**를 화면에 출력하도록 설정하는 전역 옵션

In [5]:
from langchain_classic import hub
from langchain_classic.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰줘."})



> Entering new AgentExecutor chain...
Thought: sample_data 디렉터리에 있는 파일 목록을 확인하기 위해 터미널 명령어를 사용하겠습니다. 인코딩 문제를 방지하기 위해 PowerShell에서 UTF-8 인코딩을 설정한 후 파일 목록을 확인하겠습니다.
Action: terminal
Action Input: chcp 65001Executing command:
 chcp 65001
Active code page: 65001


c:\ProgramData\anaconda3\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Thought: UTF-8 인코딩으로 변경되었으니, sample_data 디렉터리의 파일 목록을 확인하는 명령어를 실행하겠습니다.
Action: terminal
Action Input: dir sample_dataExecuting command:
 dir sample_data
 Volume in drive C has no label.
 Volume Serial Number is 6C6F-4BFC

 Directory of c:\SeSAC_AI_3rd\study_python\AI\LLM

File Not Found
Thought: sample_data 디렉터리가 현재 경로에 없거나 경로가 다를 수 있습니다. 현재 작업 디렉터리를 확인하고, sample_data 디렉터리가 존재하는지 확인하겠습니다.
Action: terminal
Action Input: cdExecuting command:
 cd
c:\SeSAC_AI_3rd\study_python\AI\LLM
Thought: 현재 작업 디렉터리는 c:\SeSAC_AI_3rd\study_python\AI\LLM 입니다. 이 위치에서 sample_data 디렉터리가 있는지 확인하기 위해 상위 디렉터리 목록을 확인하겠습니다.
Action: terminal
Action Input: dir ..\sample_dataExecuting command:
 dir ..\sample_data
 Volume in drive C has no label.
 Volume Serial Number is 6C6F-4BFC

 Directory of c:\SeSAC_AI_3rd\study_python\AI

File Not Found
Thought: sample_data 디렉터리가 현재 경로와 상위 경로 모두에 없으므로, 더 상위 경로에서 sample_data 디렉터리를 찾아보겠습니다. c:\SeSAC_AI_3rd\study_python 경로에서 sample_data 디렉터리가 있는지 확인하겠습니다.

Action: terminal
Actio

In [8]:
print(type(result), type(result["output"]))
print(result["input"])
print(result["output"])

<class 'dict'> <class 'str'>
sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰줘.
sample_data 디렉터리는 C:\ProgramData\anaconda3\Lib\site-packages\matplotlib\mpl-data\sample_data 경로에 있으며, 해당 디렉터리 내 파일 목록은 다음과 같습니다.

- data_x_x2_x3.csv
- eeg.dat
- embedding_in_wx3.xrc
- goog.npz
- grace_hopper.jpg
- jacksboro_fault_dem.npz
- logo2.png
- membrane.dat
- Minduka_Present_Blue_Pack.png
- msft.csv
- README.txt
- s1045.ima.gz
- Stocks.csv
- topobathy.npz

그리고 하위 디렉터리로는 axes_grid가 있습니다.


### Tools

In [9]:
from langchain_classic.tools import Tool

def my_super_func(param):
    return "42"

tools = [
    Tool.from_function(
        func=my_super_func,
        name="The_Answer",
        description="생명, 우주, 그리고 모든 것에 대한 궁극적인 질문의 답"
    )
]

In [11]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "이 세계의 진리를 알려주세요"})
print(result)



> Entering new AgentExecutor chain...
Question: 이 세계의 진리를 알려주세요
Thought: 이 세계의 진리에 대한 궁극적인 답을 찾기 위해 The_Answer 도구를 사용해야 한다.
Action: The_Answer
Action Input: 이 세계의 진리42Thought: The observation "42" is a reference to the famous answer to the ultimate question of life, the universe, and everything from Douglas Adams' "The Hitchhiker's Guide to the Galaxy." This suggests that the ultimate truth or answer to the question about the world's truth is symbolically "42."
Final Answer: 이 세계의 진리에 대한 궁극적인 답은 "42"입니다.

> Finished chain.
{'input': '이 세계의 진리를 알려주세요', 'output': '이 세계의 진리에 대한 궁극적인 답은 "42"입니다.'}


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import PromptTemplate
from langchain_classic import LLMChain

summarize_template = """아래의 글을 결론만 한 마디로 요약해 주세요.
{input}
"""
summarize_prompt = PromptTemplate(input_variables=["input"], template=summarize_template)
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)
tools = [
    Tool.from_function(
        func=summarize_chain.run,
        name="Summarizer",
        description="Text summarizer"
    )
]

C:\Users\User\AppData\Local\Temp\ipykernel_12668\2395266583.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)


In [14]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
text = """다음을 요약해 주세요.
안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다.
OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다.
저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다.
하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다."""
result = agent_executor.invoke({"input": text})



> Entering new AgentExecutor chain...
Thought: The user wants a summary of the given Korean text. I will use the Summarizer tool to create a concise summary.

Action: Summarizer
Action Input: 안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다. 제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다. 질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.저는 다양한 주제에 대해 대화하고 질문에 답할 수 있는 AI 언어 모델입니다.Question: 다음을 요약해 주세요.  
안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다.  
OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다.  
저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.  
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다.  
하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.  
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.

Thought: The user wants a summary of the given Korean text. I will use the Summarizer tool to create a concise summary.

Action: Summarizer  
Action Input: 안녕하세요! 저

In [15]:
print(result["output"])

저는 다양한 주제에 대해 대화하고 질문에 답할 수 있는 AI 언어 모델입니다.


### Function calling을 사용하는 OpenAI Functions Agent

In [16]:
from langchain_classic.agents import AgentExecutor, create_openai_functions_agent
from langchain_classic.agents import load_tools
from langchain_classic import hub
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰서 해줘."})



> Entering new AgentExecutor chain...

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir


c:\ProgramData\anaconda3\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir', 'sample_data']}`


Executing command:
 ['chcp 65001', 'dir', 'sample_data']
Parameter format not correct - 65001;dir;sample_data

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001'

{'input': 'sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰서 해줘.',
 'output': 'Agent stopped due to iteration limit or time limit.'}

### 여러 도구를 한 번에 사용

In [ ]:
# !pip install duckduckgo_search

In [18]:
import langchain
langchain.debug = True
langchain.verbose = False

In [ ]:
from duckduckgo_search import DuckDuckGoSearchRun
from langchain_classic.agents import AgentExecutor, create_openai_functions_agent
from langchain_classic.agents import load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "서울과 부산의 날씨를 알려줘"})

In [ ]:
print(result["output"])